In [1]:
import torch
import numpy as np

from trainer import load_embeddings
from lib.tokenizer import preprocessor
from lib.config import MODEL_EC, DEVICE
from lib.data_utils import vectorize

Running on: cuda
Reading twitter_2018 - 1grams ...
/home/houyu/.ekphrasis/stats/twitter_2018/counts_1grams.txt
Reading twitter_2018 - 2grams ...
/home/houyu/.ekphrasis/stats/twitter_2018/counts_2grams.txt
Reading twitter_2018 - 1grams ...
/home/houyu/.ekphrasis/stats/twitter_2018/counts_1grams.txt


In [11]:
def sigmoid(x):
    s = 1 / (1 + np.exp(-x))
    return s

In [27]:
labels = ['anger', 'anticipation', 'disgust', 'fear', 'joy', 'love', 'optimism', 'pessimism', 'sadness', 'surprise', 'trust', 'neutral']
model_path = '/home/houyu/learning/FinalProject/out/model/EmotionClassification_0.5900_2019-05-06_00:51.model'
model_conf = MODEL_EC
max_length = 65  # 85 train 65 dev 58 test
dev_path = '/home/houyu/learning/FinalProject/datasets/new/E-c-En-dev.txt'

# Loading model
model = torch.load(model_path)
model.eval()
word2idx, _, embeddings = load_embeddings(model_conf)

head = 1
cnt = 0
with open(dev_path, 'r') as fin:
    for line in fin.readlines():
        if head == 1:
            head = 0
            continue
        
        items = line[:len(line)-1].split('\t')
        
        input_text = items[1]

        # Pre-processing inputs
        pro_sent = preprocessor(input_text)

        # Embedding and vectorize
        sample = vectorize(pro_sent, word2idx, max_length)

        # Processing to get model inputs
        samples = []
        lengths = []
        samples.append(sample)
        lengths.append(len(pro_sent))

        samples = np.asarray(samples)
        lengths = np.asarray(lengths)

        samples = torch.tensor(samples)
        lengths = torch.tensor(lengths)

        samples = samples.to(DEVICE)
        lengths = lengths.to(DEVICE)


        # Running model
        outputs, attentions = model(samples, lengths)

        posts = outputs.data.cpu().numpy()
        # predicted = np.clip(np.sign(posts), a_min=0, a_max=None)    # 1   1   0   0   0   0   1   0   0   0   1
        # predicted = predicted.astype(np.int32)
        
        pro_sigmoid = sigmoid(posts)
        predicted = np.clip(np.sign(pro_sigmoid - 0.5), a_min=0, a_max=None)
        predicted = predicted.astype(np.int32)
        
        gold_labels = []
        for item in items[2:13]:
            gold_labels.append(int(item))
        
        gold_labels = np.asarray(gold_labels)
        
        acc = 0
        and_ = (predicted & gold_labels).sum()
        or_ = (predicted | gold_labels).sum()
        if(or_ != 0):
            acc = and_/or_
        
        # print(predicted)
        # print(gold_labels)
        # print(acc)
        
        # pro_sigmoid = sigmoid(posts)
        
        # break
        if acc >= 0.98:            # <= 0.5 错误分析
            print(input_text)
            print(pro_sent)
            print(attentions)
            print(posts)
            print(pro_sigmoid)
            print(gold_labels)
            cnt += 1

cnt

Loading word embeddings: ntua_twitter_affect_310 ...
Loaded word embeddings from cache.
@RanaAyyub @rajnathsingh Oh, hidden revenge and anger...I rememberthe time,she rebutted you.
['<user>', '<user>', 'oh', ',', 'hidden', 'revenge', 'and', 'anger', '.', '<repeated>', 'i', 'rememberthe', 'time', ',', 'she', 'rebutted', 'you', '.']
tensor([0.0555, 0.0556, 0.0556, 0.0556, 0.0556, 0.0556, 0.0556, 0.0556, 0.0556,
        0.0556, 0.0556, 0.0556, 0.0556, 0.0556, 0.0556, 0.0556, 0.0556, 0.0556],
       device='cuda:0', grad_fn=<DivBackward1>)
[ 5.34075308 -3.53363538  0.70751953 -4.59115553 -3.69499111 -7.09885406
 -1.5437305  -2.96987367 -1.66402376 -5.51040745 -4.49072266]
[  9.95230615e-01   2.83702072e-02   6.69852853e-01   1.00393230e-02
   2.42452361e-02   8.25369207e-04   1.75993621e-01   4.88055870e-02
   1.59222588e-01   4.02816758e-03   1.10882111e-02]
[1 0 1 0 0 0 0 0 0 0 0]
if not then #teamchristine bc all tana has done is provoke her by tweeting shady shit and trying to be a har

Time to #impeachtrump . He is #crazy #mentally #sick and spews #hatred . #America is in trouble with him having #access to #nuclearcodes 👎👎
['time', 'to', '<hashtag>', 'impeach', 'trump', '</hashtag>', '.', 'he', 'is', '<hashtag>', 'crazy', '</hashtag>', '<hashtag>', 'mentally', '</hashtag>', '<hashtag>', 'sick', '</hashtag>', 'and', 'spews', '<hashtag>', 'hatred', '</hashtag>', '.', '<hashtag>', 'america', '</hashtag>', 'is', 'in', 'trouble', 'with', 'him', 'having', '<hashtag>', 'access', '</hashtag>', 'to', '<hashtag>', 'nuclear', 'codes', '</hashtag>', '👎', '👎']
tensor([0.0232, 0.0233, 0.0233, 0.0233, 0.0233, 0.0233, 0.0233, 0.0233, 0.0233,
        0.0233, 0.0233, 0.0233, 0.0233, 0.0233, 0.0233, 0.0233, 0.0233, 0.0233,
        0.0233, 0.0233, 0.0233, 0.0233, 0.0233, 0.0233, 0.0233, 0.0233, 0.0233,
        0.0233, 0.0233, 0.0233, 0.0233, 0.0233, 0.0233, 0.0233, 0.0233, 0.0233,
        0.0233, 0.0233, 0.0233, 0.0233, 0.0233, 0.0233, 0.0233],
       device='cuda:0', grad_fn=<DivBackwa

tensor([0.0714, 0.0714, 0.0714, 0.0714, 0.0714, 0.0714, 0.0714, 0.0714, 0.0714,
        0.0714, 0.0714, 0.0714, 0.0714, 0.0714],
       device='cuda:0', grad_fn=<DivBackward1>)
[-1.55473423 -5.10378313 -1.02772295 -4.02210665 -4.87402153 -6.01801348
 -4.51133156  0.61613381  6.9780736  -5.20080948 -8.45963669]
[  1.74403548e-01   6.03705784e-03   2.63525784e-01   1.75998788e-02
   7.58460304e-03   2.42858846e-03   1.08644906e-02   6.49338722e-01
   9.99068797e-01   5.48188388e-03   2.11804145e-04]
[0 0 0 0 0 0 0 1 1 0 0]
@theIeansquad @SatanHeavenly Rap is so unbearable and horrific.
['<user>', '<user>', 'rap', 'is', 'so', 'unbearable', 'and', 'horrific', '.']
tensor([0.1111, 0.1111, 0.1111, 0.1111, 0.1111, 0.1111, 0.1111, 0.1111, 0.1111],
       device='cuda:0', grad_fn=<DivBackward1>)
[  3.07567644  -5.29828024   3.33525348   0.6329574   -4.79601526
  -8.73671818  -6.57014656  -3.48405051   0.31553778  -5.44050169
 -10.12337589]
[  9.55878139e-01   4.97530820e-03   9.65618610e-01   6

@1720maryknoll I was #fuming Kenny.
['<user>', 'i', 'was', '<hashtag>', 'fuming', '</hashtag>', 'kenny', '.']
tensor([0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250],
       device='cuda:0', grad_fn=<DivBackward1>)
[ 6.81591654 -5.51835108  1.01680493 -6.84631062 -3.14197516 -8.37659931
 -4.79171848 -4.94006443 -1.51916957 -5.55741072 -8.51368999]
[  9.98905063e-01   3.99642298e-03   7.34349787e-01   1.06224208e-03
   4.14086469e-02   2.30138452e-04   8.22989177e-03   7.10331928e-03
   1.79583833e-01   3.84392240e-03   2.00661598e-04]
[1 0 1 0 0 0 0 0 0 0 0]
I've got builders in my office and I have a game to make. Perhaps ill start sketching the next game... #gamedev #indiedev 
['i', 'have', 'got', 'builders', 'in', 'my', 'office', 'and', 'i', 'have', 'a', 'game', 'to', 'make', '.', 'perhaps', 'ill', 'start', 'sketching', 'the', 'next', 'game', '.', '<repeated>', '<hashtag>', 'game', 'dev', '</hashtag>', '<hashtag>', 'indie', 'dev', '</hashtag>']
tensor([0.0312, 0.0312

Don't fucking tag me in pictures as 'family first' when you cut me out 5 years ago. You're no one to me. 
['do', 'not', 'fucking', 'tag', 'me', 'in', 'pictures', 'as', "'", 'family', 'first', "'", 'when', 'you', 'cut', 'me', 'out', '<number>', 'years', 'ago', '.', 'you', 'are', 'no', 'one', 'to', 'me', '.']
tensor([0.0357, 0.0357, 0.0357, 0.0357, 0.0357, 0.0357, 0.0357, 0.0357, 0.0357,
        0.0357, 0.0357, 0.0357, 0.0357, 0.0357, 0.0357, 0.0357, 0.0357, 0.0357,
        0.0357, 0.0357, 0.0357, 0.0357, 0.0357, 0.0357, 0.0357, 0.0357, 0.0357,
        0.0357], device='cuda:0', grad_fn=<DivBackward1>)
[ 4.60967684 -4.19716597  3.54804707 -4.52396917 -4.02709198 -6.84164286
 -5.03904247 -2.86772084 -0.99202067 -4.66673088 -7.58894968]
[  9.90143061e-01   1.48153407e-02   9.72024322e-01   1.07295178e-02
   1.75138880e-02   1.06720661e-03   6.43823063e-03   5.37724979e-02
   2.70513147e-01   9.31536686e-03   5.05756354e-04]
[1 0 1 0 0 0 0 0 0 0 0]
Kid at camp said my inner thigh looked like

...

+ cant get to tell them what will offend them or not
['+', 'cant', 'get', 'to', 'tell', 'them', 'what', 'will', 'offend', 'them', 'or', 'not']
tensor([0.0833, 0.0833, 0.0833, 0.0833, 0.0833, 0.0833, 0.0833, 0.0833, 0.0833,
        0.0833, 0.0833, 0.0833], device='cuda:0', grad_fn=<DivBackward1>)
[ 1.74839294 -1.63013113  1.77538443 -2.70592046 -3.65977812 -6.55398941
 -2.38362479 -1.57551968 -0.51438361 -3.00276804 -4.35873985]
[ 0.85175002  0.1638124   0.85512596  0.0626249   0.02509239  0.0014224
  0.08442994  0.17143093  0.37416646  0.04730098  0.01263287]
[1 0 1 0 0 0 0 0 0 0 0]
A trip to the dentist never gets any easier!! #nervous
['a', 'trip', 'to', 'the', 'dentist', 'never', 'gets', 'any', 'easier', '!', '<repeated>', '<hashtag>', 'nervous', '</hashtag>']
tensor([0.0714, 0.0714, 0.0714, 0.0714, 0.0714, 0.0714, 0.0714, 0.0714, 0.0714,
        0.0714, 0.0714, 0.0714, 0.0714, 0.0714],
       device='cuda:0', grad_fn=<DivBackward1>)
[-4.99224424 -1.86754286 -3.98271084  7.48628759 

@0510Keisuke devour the unborn\nhuman rejection\nfrom wrath to ruins
['<user>', 'devour', 'the', 'unborn', '\\', 'nhuman', 'rejection', '\\', 'nfrom', 'wrath', 'to', 'ruins']
tensor([0.0833, 0.0833, 0.0833, 0.0833, 0.0833, 0.0833, 0.0833, 0.0833, 0.0833,
        0.0833, 0.0833, 0.0833], device='cuda:0', grad_fn=<DivBackward1>)
[ 1.60728741 -3.73549533  1.59067976 -2.07552528 -5.81047678 -8.95082951
 -4.46336555 -2.29794669 -0.85282177 -7.44650316 -7.69842005]
[  8.33034456e-01   2.33052541e-02   8.30711663e-01   1.11498497e-01
   2.98705208e-03   1.29612788e-04   1.13922367e-02   9.12931636e-02
   2.98841268e-01   5.83138084e-04   4.53337590e-04]
[1 0 1 0 0 0 0 0 0 0 0]
Each of us have varied #talents #gifts &amp; #abilities If we #all #work #together #we can #help #make the #world a #better &amp; #brighter #place
['each', 'of', 'us', 'have', 'varied', '<hashtag>', 'talents', '</hashtag>', '<hashtag>', 'gifts', '</hashtag>', '&', '<hashtag>', 'abilities', '</hashtag>', 'if', 'we', '<ha

#CNN really needs to get out of the #Propaganda Business.. 30 seconds on USN fallen Soldiers tragedy. Right back at spewing #hatred #POTUS
['<hashtag>', 'cnn', '</hashtag>', 'really', 'needs', 'to', 'get', 'out', 'of', 'the', '<hashtag>', 'propaganda', '</hashtag>', 'business', '.', '<repeated>', '<number>', 'seconds', 'on', '<allcaps>', 'usn', '</allcaps>', 'fallen', 'soldiers', 'tragedy', '.', 'right', 'back', 'at', 'spewing', '<hashtag>', 'hatred', '</hashtag>', '<hashtag>', 'potus', '</hashtag>']
tensor([0.0041, 0.0041, 0.0041, 0.0299, 0.0299, 0.0299, 0.0299, 0.0299, 0.0299,
        0.0299, 0.0299, 0.0299, 0.0299, 0.0299, 0.0299, 0.0299, 0.0299, 0.0299,
        0.0299, 0.0299, 0.0299, 0.0299, 0.0299, 0.0299, 0.0299, 0.0299, 0.0299,
        0.0299, 0.0299, 0.0299, 0.0299, 0.0299, 0.0299, 0.0299, 0.0299, 0.0299],
       device='cuda:0', grad_fn=<DivBackward1>)
[  5.75143194  -4.66445589   4.66560316  -5.82245302  -5.61492729
  -9.86143875  -6.21788216  -2.79893088  -0.08276093  -6.37

tensor([0.0098, 0.0354, 0.0354, 0.0354, 0.0354, 0.0354, 0.0354, 0.0354, 0.0354,
        0.0354, 0.0354, 0.0354, 0.0354, 0.0354, 0.0354, 0.0354, 0.0354, 0.0354,
        0.0354, 0.0354, 0.0354, 0.0354, 0.0354, 0.0354, 0.0354, 0.0354, 0.0354,
        0.0354, 0.0354], device='cuda:0', grad_fn=<DivBackward1>)
[ 2.961658   -3.7570343   2.73040891 -3.63784528  0.27583912 -5.12031841
 -2.53359866 -3.38297701 -2.09199739 -3.79381323 -5.99638605]
[ 0.95081162  0.02281998  0.93879735  0.02563455  0.56852585  0.00593864
  0.0735361   0.03283173  0.10987707  0.02201407  0.00248155]
[1 0 1 0 1 0 0 0 0 0 0]
I've never been so excited to start a semester!
['i', 'have', 'never', 'been', 'so', 'excited', 'to', 'start', 'a', 'semester', '!']
tensor([0.0909, 0.0909, 0.0909, 0.0909, 0.0909, 0.0909, 0.0909, 0.0909, 0.0909,
        0.0909, 0.0909], device='cuda:0', grad_fn=<DivBackward1>)
[-5.06694555  0.90189379 -5.82690668 -0.54998863  2.90805626 -2.28185582
  0.63985127 -4.8374505  -5.41293907 -2.07762885

252

In [23]:
cnt # acc >= 0.9; 0.5 245, 0.3 191 => first model, # new model 252

191

In [20]:
# 448 <= 0.5

In [25]:
a = {1:[]}

In [26]:
a = set()
a.add(1)
a.add(1)
a

{1}